In [1]:
import sys
import os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '../..')))

# Path Configuration
from tools.preprocess import *

# Processing context
trait = "Celiac_Disease"
cohort = "GSE106260"

# Input paths
in_trait_dir = "../../input/GEO/Celiac_Disease"
in_cohort_dir = "../../input/GEO/Celiac_Disease/GSE106260"

# Output paths
out_data_file = "../../output/preprocess/Celiac_Disease/GSE106260.csv"
out_gene_data_file = "../../output/preprocess/Celiac_Disease/gene_data/GSE106260.csv"
out_clinical_data_file = "../../output/preprocess/Celiac_Disease/clinical_data/GSE106260.csv"
json_path = "../../output/preprocess/Celiac_Disease/cohort_info.json"


### Step 1: Initial Data Loading

In [2]:
from tools.preprocess import *
# 1. Identify the paths to the SOFT file and the matrix file
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)

# 2. Read the matrix file to obtain background information and sample characteristics data
background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']
background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)

# 3. Obtain the sample characteristics dictionary from the clinical dataframe
sample_characteristics_dict = get_unique_values_by_row(clinical_data)

# 4. Explicitly print out all the background information and the sample characteristics dictionary
print("Background Information:")
print(background_info)
print("Sample Characteristics Dictionary:")
print(sample_characteristics_dict)


Background Information:
!Series_title	"Immunopathology of childhood celiac disease-Key role of intestinal epithelial cells"
!Series_summary	"This SuperSeries is composed of the SubSeries listed below."
!Series_overall_design	"Refer to individual Series"
Sample Characteristics Dictionary:
{0: ['cell line: colon carcinoma cell line T84'], 1: ['treatment: CTR', 'treatment: A. graevenitzii', 'treatment: bacteria mix', 'treatment: bacteria mix with gluten', 'treatment: L. umeaense', 'treatment: P. jejuni (isolates CD3:28)', 'treatment: P. jejuni (isolates CD3:27)', 'treatment: Gluten', 'treatment: bacteria mix + Gluten']}


### Step 2: Gene Data Extraction

In [3]:
# 1. Get the SOFT and matrix file paths again 
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)
print(f"Matrix file found: {matrix_file}")

# 2. Use the get_genetic_data function from the library to get the gene_data
try:
    gene_data = get_genetic_data(matrix_file)
    print(f"Gene data shape: {gene_data.shape}")
    
    # 3. Print the first 20 row IDs (gene or probe identifiers)
    print("First 20 gene/probe identifiers:")
    print(gene_data.index[:20])
except Exception as e:
    print(f"Error extracting gene data: {e}")


Matrix file found: ../../input/GEO/Celiac_Disease/GSE106260/GSE106260-GPL10558_series_matrix.txt.gz
Gene data shape: (47230, 36)
First 20 gene/probe identifiers:
Index(['ILMN_1343291', 'ILMN_1343295', 'ILMN_1651199', 'ILMN_1651209',
       'ILMN_1651210', 'ILMN_1651221', 'ILMN_1651228', 'ILMN_1651229',
       'ILMN_1651230', 'ILMN_1651232', 'ILMN_1651235', 'ILMN_1651236',
       'ILMN_1651237', 'ILMN_1651238', 'ILMN_1651249', 'ILMN_1651253',
       'ILMN_1651254', 'ILMN_1651259', 'ILMN_1651260', 'ILMN_1651262'],
      dtype='object', name='ID')


### Step 3: Gene Identifier Review

In [4]:
# Identifying the gene identifiers
# These are ILMN identifiers from Illumina microarray platforms
# They are not standard human gene symbols and need to be mapped
# ILMN_XXXXXXX is the Illumina BeadChip array ID format that needs mapping to gene symbols

requires_gene_mapping = True


### Step 4: Gene Annotation

In [5]:
# 1. Use the 'get_gene_annotation' function from the library to get gene annotation data from the SOFT file.
gene_annotation = get_gene_annotation(soft_file)

# 2. Use the 'preview_df' function from the library to preview the data and print out the results.
print("Gene annotation preview:")
print(preview_df(gene_annotation))


Gene annotation preview:
{'ID': ['ILMN_1722532', 'ILMN_1708805', 'ILMN_1672526', 'ILMN_1703284', 'ILMN_2185604'], 'Species': ['Homo sapiens', 'Homo sapiens', 'Homo sapiens', 'Homo sapiens', 'Homo sapiens'], 'Source': ['RefSeq', 'RefSeq', 'RefSeq', 'RefSeq', 'RefSeq'], 'Search_Key': ['ILMN_25544', 'ILMN_10519', 'ILMN_17234', 'ILMN_502', 'ILMN_19244'], 'Transcript': ['ILMN_25544', 'ILMN_10519', 'ILMN_17234', 'ILMN_502', 'ILMN_19244'], 'ILMN_Gene': ['JMJD1A', 'NCOA3', 'LOC389834', 'SPIRE2', 'C17ORF77'], 'Source_Reference_ID': ['NM_018433.3', 'NM_006534.2', 'NM_001013655.1', 'NM_032451.1', 'NM_152460.2'], 'RefSeq_ID': ['NM_018433.3', 'NM_006534.2', 'NM_001013655.1', 'NM_032451.1', 'NM_152460.2'], 'Entrez_Gene_ID': [55818.0, 8202.0, 389834.0, 84501.0, 146723.0], 'GI': [46358420.0, 32307123.0, 61966764.0, 55749599.0, 48255961.0], 'Accession': ['NM_018433.3', 'NM_006534.2', 'NM_001013655.1', 'NM_032451.1', 'NM_152460.2'], 'Symbol': ['JMJD1A', 'NCOA3', 'LOC389834', 'SPIRE2', 'C17orf77'], 'Prot

### Step 5: Gene Identifier Mapping

In [6]:
# 1. Determine the mapping between gene identifiers and gene symbols
# From the annotation preview, we can see that:
# - The 'ID' column contains the ILMN probe identifiers (which match gene_data.index)
# - The 'Symbol' column contains the gene symbols we want to map to

# 2. Get the gene mapping dataframe
gene_mapping = get_gene_mapping(gene_annotation, prob_col='ID', gene_col='Symbol')
print(f"Gene mapping dataframe shape: {gene_mapping.shape}")
print("First 5 rows of gene mapping:")
print(gene_mapping.head())

# 3. Apply the gene mapping to convert probe-level data to gene-level expression data
gene_data = apply_gene_mapping(gene_data, gene_mapping)
print(f"Mapped gene expression data shape: {gene_data.shape}")
print("First 10 gene symbols in the mapped data:")
print(gene_data.index[:10])

# Normalize the gene symbols to handle synonyms and variants
gene_data = normalize_gene_symbols_in_index(gene_data)
print(f"After normalization, gene expression data shape: {gene_data.shape}")
print("First 10 normalized gene symbols:")
print(gene_data.index[:10])


Gene mapping dataframe shape: (24526, 2)
First 5 rows of gene mapping:
             ID       Gene
0  ILMN_1722532     JMJD1A
1  ILMN_1708805      NCOA3
2  ILMN_1672526  LOC389834
3  ILMN_1703284     SPIRE2
4  ILMN_2185604   C17orf77
Mapped gene expression data shape: (17824, 36)
First 10 gene symbols in the mapped data:
Index(['A1BG', 'A2BP1', 'A2M', 'A2ML1', 'A3GALT2', 'A4GALT', 'A4GNT', 'AAA1',
       'AAAS', 'AACS'],
      dtype='object', name='Gene')


After normalization, gene expression data shape: (16856, 36)
First 10 normalized gene symbols:
Index(['A1BG', 'A2M', 'A2ML1', 'A3GALT2', 'A4GALT', 'A4GNT', 'AAA1', 'AAAS',
       'AACS', 'AADAC'],
      dtype='object', name='Gene')


### Step 6: Data Normalization and Linking

In [7]:
# 1. Gene data is already normalized from previous step - no need to normalize again
# Save the normalized gene data
os.makedirs(os.path.dirname(out_gene_data_file), exist_ok=True)
gene_data.to_csv(out_gene_data_file)
print(f"Normalized gene data saved to {out_gene_data_file}")

# 2. Process clinical data from the clinical dataframe we obtained in Step 1
# From the characteristics dictionary, we know we need to analyze the clinical features in detail
print("Clinical data from matrix file:")
print(clinical_data.head())

# The clinical data is sparse for this dataset as seen in Step 1
# Extract information from the sample characteristics for celiac disease analysis
# In this case, the cell type information is a proxy for trait - intestinal epithelial cells vs intraepithelial lymphocytes
def convert_cell_type(cell_type_str):
    if isinstance(cell_type_str, str):
        if 'epithelial' in cell_type_str.lower():
            return 0  # Control/normal
        elif 'lymphocytes' in cell_type_str.lower():
            return 1  # Disease/case
    return None  # For any other values or missing data

# Process the clinical data to extract trait information
# Find the row index that contains cell type information
cell_type_row = None
for idx, row_data in clinical_data.iterrows():
    row_values = list(row_data.values)
    for val in row_values:
        if isinstance(val, str) and 'cell type' in val.lower():
            cell_type_row = idx
            break
    if cell_type_row is not None:
        break

# If we found the row with cell type info, extract the trait data
if cell_type_row is not None:
    selected_clinical_data = geo_select_clinical_features(
        clinical_data,
        trait=trait,
        trait_row=cell_type_row,
        convert_trait=convert_cell_type
    )
    print("Selected clinical features:")
    print(selected_clinical_data)
else:
    # If no cell type info is found, we'll need to handle this case
    print("No cell type information found in clinical data.")
    # Create a dummy clinical dataframe with just the IDs from gene data
    selected_clinical_data = pd.DataFrame(
        index=[trait], 
        columns=gene_data.columns,
        data=[[None] * len(gene_data.columns)]
    )

# Save the processed clinical data
os.makedirs(os.path.dirname(out_clinical_data_file), exist_ok=True)
selected_clinical_data.to_csv(out_clinical_data_file)
print(f"Clinical data saved to {out_clinical_data_file}")

# 3. Link clinical and genetic data
linked_data = geo_link_clinical_genetic_data(selected_clinical_data, gene_data)
print(f"Linked data shape: {linked_data.shape}")
print("Linked data preview (first 5 rows, 5 columns):")
print(linked_data.iloc[:5, :5])

# 4. Handle missing values
linked_data = handle_missing_values(linked_data, trait)
print(f"Data shape after handling missing values: {linked_data.shape}")

# 5. Check for bias in features
is_biased, linked_data = judge_and_remove_biased_features(linked_data, trait)

# 6. Validate and save cohort information
is_usable = validate_and_save_cohort_info(
    is_final=True,
    cohort=cohort,
    info_path=json_path,
    is_gene_available=True,
    is_trait_available=True,
    is_biased=is_biased,
    df=linked_data,
    note="Dataset contains gene expression data from intestinal epithelial cells vs intraepithelial lymphocytes in Celiac Disease study."
)

# 7. Save the linked data if usable
if is_usable:
    os.makedirs(os.path.dirname(out_data_file), exist_ok=True)
    linked_data.to_csv(out_data_file)
    print(f"Linked data saved to {out_data_file}")
else:
    print("Dataset is not usable for analysis. No linked data file saved.")

Normalized gene data saved to ../../output/preprocess/Celiac_Disease/gene_data/GSE106260.csv
Clinical data from matrix file:
         !Sample_geo_accession                                GSM2753759  \
0  !Sample_characteristics_ch1  cell line: colon carcinoma cell line T84   
1  !Sample_characteristics_ch1                            treatment: CTR   

                                 GSM2753760  \
0  cell line: colon carcinoma cell line T84   
1                            treatment: CTR   

                                 GSM2753761  \
0  cell line: colon carcinoma cell line T84   
1                            treatment: CTR   

                                 GSM2753762  \
0  cell line: colon carcinoma cell line T84   
1                treatment: A. graevenitzii   

                                 GSM2753763  \
0  cell line: colon carcinoma cell line T84   
1                treatment: A. graevenitzii   

                                 GSM2753764  \
0  cell line: colon carcinoma c

/media/techt/DATA/GenoAgent/tools/preprocess.py:400: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  linked_data = pd.concat([clinical_df, genetic_df], axis=0).T
